In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import pymysql

conn = pymysql.connect(host='Localhost',user='root',password='ehfkwl1!',db='testdb',charset='utf8')
cursor = conn.cursor(pymysql.cursors.DictCursor)

In [3]:
sql1 = "SELECT * FROM `tb_comment`;" #테이블명 적기
cursor.execute(sql1)
result = cursor.fetchall()

In [4]:
data1 = pd.DataFrame(result)
data1.loc[451216]

ID                       451217
NewsID                   489710
UserID                   394234
WritedAt    2022-11-09 03:08:32
Content            이게 기사야  기래기야
Name: 451216, dtype: object

In [5]:
data1 = data1.drop('ID', axis=1)


In [6]:
data1.rename(columns = {'NewsID' : 'ID'}, inplace = True)
data1

,ID,UserID,WritedAt,Content
0,376816,359619,0000-00-00 00:00:00,정말 뭣같네..물가잡는다고 돈회수한다고 고금리정책쓰면서 다시 돈을 풀어? 정신나간것...
1,376817,365124,0000-00-00 00:00:00,주둥아리만 나불나불
2,376817,358144,0000-00-00 00:00:00,진태양난때는 모든 부처가 산업 주무부처가 되더니 오늘부터는 모든 부처가 안전 주무부처냐?
3,376817,363901,0000-00-00 00:00:00,'안전을 중시하는 관료적인 사고는 버려야한다' by 6.22 굥..수괴가 이따위인데...
4,376817,257844,0000-00-00 00:00:00,ㅋㅋㅋ 돌아 버리것네~~\n얼마전에 안전을 중시하는 관료적인 사고를 버리라는 넘이 ...
...,...,...,...,...
2504498,1246190,1058326,2022-11-24 00:00:00,대~mbc ~~~~~
2504499,1246190,1058327,2022-11-24 00:00:00,전두환이가 가창 조아할 상황이구만..개돼지들에게는 3s가 딱이지 ㅋㅋ
2504500,1246190,1058328,2022-11-24 00:00:00,안봐요. 안응원해요.
2504501,1246190,1058329,2022-11-24 00:00:00,뱃살에 기름기 잔뜩껴라


In [7]:
sql2 = "SELECT * FROM `tb_news`;" #테이블명 적기
cursor.execute(sql2)
result2 = cursor.fetchall()

In [30]:
data2 = pd.DataFrame(result2, columns=['ID', 'Title', 'URL','MainCategory'])
data2

data2['view'] = 1
data2

,ID,Title,URL,MainCategory,view
0,1,휴스턴 레전드의 일침 “야구는 컴퓨터가 아니라 사람이 하는 것”,https://sports.news.naver.com/news?oid=410&aid...,스포츠,1
1,2,[GD레슨] 진정한 실력자가 되기 위해 필요한 세 가지 기술 ②,https://sports.news.naver.com/news?oid=435&aid...,스포츠,1
2,3,‘1무 1패’ 한국과 동병상련…카메룬 특급도 월드컵 시청 “지지 않아서 좋았다” [...,https://sports.news.naver.com/news?oid=109&aid...,스포츠,1
3,4,[삼성화재배 AI와 함께하는 바둑 해설] 흑의 의도를 비틀다,https://sports.news.naver.com/news?oid=025&aid...,스포츠,1
4,5,"어제의 적이 동지? 벌랜더, 다저스 만난다",https://sports.news.naver.com/news?oid=410&aid...,스포츠,1
...,...,...,...,...,...
1201936,1335604,"HK직캠｜첫사랑, 열일곱 소녀들의 솔직한 사랑 표현…'러브티콘' 무대",https://v.daum.net/v/20221117234001184,entertain,1
1201937,1335605,"에이핑크 정은지, 12월 단독 콘서트 개최 [공식]",https://v.daum.net/v/20221117234507221,entertain,1
1201938,1335606,KBS 설 특집 송골매 콘서트…이선균·장기하 등 출연,https://v.daum.net/v/20221117174810728,entertain,1
1201939,1335607,"에이티즈, 팬클럽 창단 4주년 맞아 ‘트레저’ 시리즈 재발매",https://v.daum.net/v/20221117174813729,entertain,1


In [31]:
data2 = data2[data2['MainCategory']=='IT']
data2

,ID,Title,URL,MainCategory,view
451216,584884,"레드브릭, 지스타 현장서 C2E 모델 첫 공개",https://v.daum.net/v/20221117142105685,IT,1
451217,584885,"리걸테크㈜, 자사 VDR 솔루션 내달 일본 시장 진출",https://v.daum.net/v/20221117090125492,IT,1
451218,584886,英국제통상부가 ‘삼쩜삼’ 자비스를 방문한 이유는?,https://v.daum.net/v/20221117090008375,IT,1
451219,584887,"야놀자, 글로벌 여행 관련 시상식서 잇따라 수상",https://v.daum.net/v/20221117085005834,IT,1
451220,584888,"인기협, 21일 '온라인 플랫폼 규제의 올바른 방향성' 토론회 개최",https://v.daum.net/v/20221117090431689,IT,1
...,...,...,...,...,...
472985,606653,"쉽고 재미있는 요리 앱 `버터얌`, `코엑스 푸드위크` 참여",https://v.daum.net/v/20221102110514025,IT,1
472986,606654,"스파이더, ‘코리아 푸드테크 산업전’ 참가",https://v.daum.net/v/20221102110801174,IT,1
472987,606655,"혹독한 겨울, '버티기'에 들어간 스타트업",https://v.daum.net/v/20221102101752030,IT,1
472988,606656,[알림]우영우 성공스토리부터 SKT 웹3 전략까지…16일 ECF 2022서 공개,https://v.daum.net/v/20221102102247215,IT,1


In [35]:
merge_df = pd.merge(data1, data2, on='ID')
merge_df

,ID,UserID,WritedAt,Content,Title,URL,MainCategory,view
0,593046,460575,2022-11-25 18:53:50,잘하고 못하고를 떠나쫄깃한 무승부,"한국vs우루과이 0:0 무승부…""근래 봤던 한국 경기 중 최고""",https://v.daum.net/v/20221125003617254,IT,1
1,593046,494771,2022-11-25 18:09:06,내가봤을때는 엉성 그자체 정상적인 스코어면 2:1패,"한국vs우루과이 0:0 무승부…""근래 봤던 한국 경기 중 최고""",https://v.daum.net/v/20221125003617254,IT,1
2,593046,434598,2022-11-25 18:00:45,김거늬땜 축구응원도 하기 싫어짐요!!,"한국vs우루과이 0:0 무승부…""근래 봤던 한국 경기 중 최고""",https://v.daum.net/v/20221125003617254,IT,1
3,593046,434599,2022-11-25 18:00:19,잘했지만 이강인 좀빨리 투입했어야했다. 이강인은 창조적플레이에능하다.빤히보이는패스...,"한국vs우루과이 0:0 무승부…""근래 봤던 한국 경기 중 최고""",https://v.daum.net/v/20221125003617254,IT,1
4,593046,464627,2022-11-25 17:50:35,의조는 선발에서 빼자,"한국vs우루과이 0:0 무승부…""근래 봤던 한국 경기 중 최고""",https://v.daum.net/v/20221125003617254,IT,1
...,...,...,...,...,...,...,...,...
22372,590257,461695,2022-11-15 10:10:27,지금 하는거 보면 50년전 일은 경쟁심에서 시작된 뻥이 확실함.. 지금쯤이면 쉽게 ...,유인 달 탐사 본격화할까...역대급 우주발사체 'SLS' 16일 발사,https://v.daum.net/v/20221114134013393,IT,1
22373,590031,465127,2022-11-14 22:53:03,대만의 자유를!이 말 한마디에 끝장났습니다. 내가 바봅니까? ㅠㅠ 아직도 모르겠냐...,반도체 수출액 100억달러 붕괴…석달째 줄어,https://v.daum.net/v/20221114110019824,IT,1
22374,590031,494661,2022-11-14 22:26:24,훌륭하신 대통령 각하. !! 외교의 힘입니다,반도체 수출액 100억달러 붕괴…석달째 줄어,https://v.daum.net/v/20221114110019824,IT,1
22375,590031,492823,2022-11-14 11:38:31,"윤석열이가 기업,경제,물가,집값,주가 폭락, 사람까지 죽는 이런 능력 탁월하네. 오...",반도체 수출액 100억달러 붕괴…석달째 줄어,https://v.daum.net/v/20221114110019824,IT,1


In [44]:
merge_df['view'] = merge_df['view'].fillna(0)

In [54]:
merge_data = merge_df.pivot_table('view', index='UserID', columns='Title')
merge_data

Title,"""'삼성 텃밭'도 옛말""…뉴진스가 든 애플 '아이폰' 韓 습격 빨라진다","""'아이폰 타령'하던 아이들 다 크면…"" 애플의 기막힌 노림수 [애플페이 상륙(2)]","""-1억"" 위믹스 투자자들 충격·분노·절망…일부는 ""끝까지 믿는다""","""011, 이제 진짜 안녕""…3년 만에 01X 논란 종지부","""1000년 기다려도 못볼 우주쇼"" 오늘밤 붉은달이 천왕성 가린다","""10㎞ 밖에서 1초당 1.75Gb 다운"" 삼성, 28㎓대역 5G통신 '세계 신기록'","""10년내 한반도 상공 수천만대 위성 통과…감시체계 추가해야""","""10조 갑부, 세기의 재산분할""…스마일게이트 권혁빈 이혼 소송","""28㎓는 별로지만 3.7㎓는 원해""…통신 3사 셈법 다른 까닭은 [이상은의 IT 산책]","""2년전 소말리아서 발견된 운석서 지구상 없던 광물 2종 발견""",...,화성에 우주비행사 안착 시킬 팽창식 열방패 지구 대기권서 시연,"화재 걱정 없는 전고체 배터리..""전기차엔 2030년 이후에나 활용가능""","화질 제한 트위치, 한국서 VOD 다시보기 중단한다","화질 제한한 트위치, 이번엔 한국만 VOD 콘텐츠 중단","환경부, 녹조 독소 관련 WHO 권고 무시?",활명수가 1897년생이라고? 125년 역사가 궁금한 사람 주목!,"회의록 부실 정황?…국가기록원, 게임위에 사실상 경고","휴대폰 하나로 신분인증…""삼성페이로 운전면허증 확인하세요""",힘 못쓰는 국산 코로나19 백신‧치료제… 오미크론 대응 역부족,﻿사회적 문제 해결 나선 기업들...KT 덕에 안 춥다 [IT돋보기]
UserID,,,,,,,,,,,,,,,,,,,,,
434598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
434599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
434600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
434601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
434602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
501360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
501361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
merge_data = merge_data.fillna(0)
merge_data = merge_data.T
merge_data

UserID,434598,434599,434600,434601,434602,434603,434604,434605,434606,434607,...,501326,501338,501339,501341,501350,501352,501360,501361,501367,501370
Title,,,,,,,,,,,,,,,,,,,,,
"""'삼성 텃밭'도 옛말""…뉴진스가 든 애플 '아이폰' 韓 습격 빨라진다",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""'아이폰 타령'하던 아이들 다 크면…"" 애플의 기막힌 노림수 [애플페이 상륙(2)]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""-1억"" 위믹스 투자자들 충격·분노·절망…일부는 ""끝까지 믿는다""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""011, 이제 진짜 안녕""…3년 만에 01X 논란 종지부",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""1000년 기다려도 못볼 우주쇼"" 오늘밤 붉은달이 천왕성 가린다",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
활명수가 1897년생이라고? 125년 역사가 궁금한 사람 주목!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"회의록 부실 정황?…국가기록원, 게임위에 사실상 경고",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"휴대폰 하나로 신분인증…""삼성페이로 운전면허증 확인하세요""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
merge_data.sum()

UserID
434598    1.0
434599    1.0
434600    1.0
434601    1.0
434602    1.0
         ... 
501352    2.0
501360    4.0
501361    1.0
501367    1.0
501370    1.0
Length: 16469, dtype: float64

In [70]:
cosine_sim = cosine_similarity(merge_data, merge_data)
cosine_sim.shape

(2138, 2138)

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [71]:
cosine_value = pd.DataFrame(data = cosine_sim, index=merge_data.index, columns= merge_data.index)

In [72]:
cosine_value

Title,"""'삼성 텃밭'도 옛말""…뉴진스가 든 애플 '아이폰' 韓 습격 빨라진다","""'아이폰 타령'하던 아이들 다 크면…"" 애플의 기막힌 노림수 [애플페이 상륙(2)]","""-1억"" 위믹스 투자자들 충격·분노·절망…일부는 ""끝까지 믿는다""","""011, 이제 진짜 안녕""…3년 만에 01X 논란 종지부","""1000년 기다려도 못볼 우주쇼"" 오늘밤 붉은달이 천왕성 가린다","""10㎞ 밖에서 1초당 1.75Gb 다운"" 삼성, 28㎓대역 5G통신 '세계 신기록'","""10년내 한반도 상공 수천만대 위성 통과…감시체계 추가해야""","""10조 갑부, 세기의 재산분할""…스마일게이트 권혁빈 이혼 소송","""28㎓는 별로지만 3.7㎓는 원해""…통신 3사 셈법 다른 까닭은 [이상은의 IT 산책]","""2년전 소말리아서 발견된 운석서 지구상 없던 광물 2종 발견""",...,화성에 우주비행사 안착 시킬 팽창식 열방패 지구 대기권서 시연,"화재 걱정 없는 전고체 배터리..""전기차엔 2030년 이후에나 활용가능""","화질 제한 트위치, 한국서 VOD 다시보기 중단한다","화질 제한한 트위치, 이번엔 한국만 VOD 콘텐츠 중단","환경부, 녹조 독소 관련 WHO 권고 무시?",활명수가 1897년생이라고? 125년 역사가 궁금한 사람 주목!,"회의록 부실 정황?…국가기록원, 게임위에 사실상 경고","휴대폰 하나로 신분인증…""삼성페이로 운전면허증 확인하세요""",힘 못쓰는 국산 코로나19 백신‧치료제… 오미크론 대응 역부족,﻿사회적 문제 해결 나선 기업들...KT 덕에 안 춥다 [IT돋보기]
Title,,,,,,,,,,,,,,,,,,,,,
"""'삼성 텃밭'도 옛말""…뉴진스가 든 애플 '아이폰' 韓 습격 빨라진다",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""'아이폰 타령'하던 아이들 다 크면…"" 애플의 기막힌 노림수 [애플페이 상륙(2)]",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""-1억"" 위믹스 투자자들 충격·분노·절망…일부는 ""끝까지 믿는다""",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""011, 이제 진짜 안녕""…3년 만에 01X 논란 종지부",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""1000년 기다려도 못볼 우주쇼"" 오늘밤 붉은달이 천왕성 가린다",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
활명수가 1897년생이라고? 125년 역사가 궁금한 사람 주목!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"회의록 부실 정황?…국가기록원, 게임위에 사실상 경고",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"휴대폰 하나로 신분인증…""삼성페이로 운전면허증 확인하세요""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [76]:
cosine_value['"1000년 기다려도 못볼 우주쇼" 오늘밤 붉은달이 천왕성 가린다'].sort_values(ascending=False)[1:10]

Title
빗썸 상폐 '싸이클럽'…싸이월드 "우리 코인 아냐"                  0.229416
카타르 월드컵에선 인공지능이 '오프사이드' 잡아낸다                  0.162221
"위믹스 1개라도 쓰면 공시" 실천 중인 위메이드…유통량 기준은 '모호'      0.162221
구글 "연말까지 10억명의 '지속가능한 행동' 이끌 것"               0.162221
과열로 녹아내린 지포스 RTX 4090 단자 논란…소비자 과실? 설계 문제?    0.132453
“이번에 놓치면 200년 뒤”…내일 밤 개기월식·천왕성 엄폐 동시 관측       0.093659
[영상]원전 수중해체 기술 나왔다..컴퓨터 게임하듯 클릭 한번에           0.093659
'4000억원 이더리움' 큰손 FTX 해커, 550억원어치 팔자 시세 뚝      0.086711
위믹스 상장폐지…알트코인·김치코인 전반 타격 '불가피'                0.081111
Name: "1000년 기다려도 못볼 우주쇼" 오늘밤 붉은달이 천왕성 가린다, dtype: float64

In [ ]:
from sklearn.decomposition import TruncatedSVD
SVD = TruncatedSVD(n_components=12)

In [ ]:
matrix = SVD.fit_transform(news_user)
matrix.shape

In [ ]:
corr = np.corrcoef(matrix)
corr.shape

In [ ]:

title_list = cosine_value.columns
title_list_1 = list(title_list)
ex_1 = title_list_1.index('https://v.daum.net/v/20221117142105685')
ex_2 = corr[ex_1]

list(title_list[(ex_2 >=0.9)])[:10]

In [21]:
data= data.pivot_table(index= 'UserID', columns='NewsID')

C:\Users\seoyj\AppData\Local\Temp\ipykernel_11176\1606444095.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data= data.pivot_table(index= 'UserID', columns='NewsID')
c:\anaconda\envs\mecab\lib\site-packages\pandas\core\reshape\reshape.py:126: RuntimeWarning: overflow encountered in long_scalars
  num_cells = num_rows * num_columns


IndexError: index 1392780377 is out of bounds for axis 0 with size 1392684744

NameError: name 'info' is not defined